In [2]:
# Step 1: Upload and Extract ZIP File
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()

# Assuming the file is named 'DATASET.zip', change the name if necessary
zip_file_path = 'DATASET.zip'
extract_folder = '/content/Property_data/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Step 2: Read and Merge All CSV Files with Correct Encoding
import pandas as pd

# List all files in the extracted folder
folder_path = os.path.join(extract_folder, 'Property_data')
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file with the 'ISO-8859-1' encoding and append it to the list
for file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, file), encoding='ISO-8859-1')
    dataframes.append(df)

# Merge all the DataFrames into one final DataFrame
merged_data = pd.concat(dataframes, ignore_index=True)

# Step 3: Clean the Data (Handle invalid columns, missing values, etc.)
# Drop unwanted columns (like empty or corrupt columns)
merged_data = merged_data.loc[:, ~merged_data.columns.str.contains('ÿþ', na=False)]

# Step 4: Check the final shape of the merged data
print(f"Final shape of the combined property data: {merged_data.shape}")


Saving DATASET.zip to DATASET (1).zip
Final shape of the combined property data: (14532, 23)


In [7]:
import zipfile
import os
import pandas as pd

# Unzip the dataset
with zipfile.ZipFile('DATASET.zip', 'r') as zip_ref:
    zip_ref.extractall("DATASET")

# List the contents of the extracted folder to find CSV files
extracted_files = os.listdir("DATASET")
print("Extracted Files:", extracted_files)

# Specify the folder containing CSV files you want to merge
folder_path = 'DATASET/Property_data'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file and append it to the list
for file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, file), encoding='ISO-8859-1')  # Using ISO-8859-1 to handle encoding errors
    dataframes.append(df)

# Merge all the DataFrames into one final DataFrame
merged_data = pd.concat(dataframes, ignore_index=True)

# Step 3: Clean the Data (Handle invalid columns, missing values, etc.)
# Drop unwanted columns (like empty or corrupt columns)
merged_data = merged_data.loc[:, ~merged_data.columns.str.contains('ÿþ', na=False)]

# Clean any unnecessary columns
merged_data_cleaned = merged_data.dropna(axis=1, how='all')

# Display the final shape of the cleaned merged data
print(f"Final shape of the combined property data: {merged_data_cleaned.shape}")


Extracted Files: ['property_interactions.csv', 'Property_data', 'property_photos.tsv']
Final shape of the combined property data: (14532, 23)


In [ ]:
hsr_layout_percentage = (property_data[property_data['locality'] == 'HSR Layout'].shape[0] / property_data.shape[0]) * 100
print(f"Percentage of properties in HSR Layout: {round(hsr_layout_percentage)}%")


In [ ]:
# Task 3: Number of unique locations
unique_locations = property_data['locality'].nunique()
print(f"Number of unique locations: {unique_locations}")


In [ ]:
# Task 4: Locality with highest average rent
highest_avg_rent_locality = property_data.groupby('locality')['rent'].mean().idxmax()
print(f"Locality with highest average rent: {highest_avg_rent_locality}")


In [ ]:
# Task 5: Feature with highest correlation with rent
correlation_with_rent = property_data.corr()['rent'].drop('rent')
highest_corr_feature = correlation_with_rent.idxmax()
print(f"Feature with highest correlation with rent: {highest_corr_feature}")


In [ ]:
import pandas as pd

# Assuming interaction_data and property_data are already loaded
interaction_data = pd.read_csv('interaction_data.csv')
property_data['activation_date'] = pd.to_datetime(property_data['activation_date'])
interaction_data['request_date'] = pd.to_datetime(interaction_data['request_date'])

# Merge interaction data with property data
merged_data = pd.merge(interaction_data, property_data, on='property_id')

# Filter interactions that happened within 7 days of activation
merged_data['interaction_within_7_days'] = (merged_data['request_date'] - merged_data['activation_date']).dt.days <= 7
interaction_counts_within_7 = merged_data.groupby('property_id')['interaction_within_7_days'].sum()

# Most common interaction count
most_common_interaction_count = interaction_counts_within_7.value_counts().idxmax()
print(f"Total number of interactions received by the majority of the properties within 7 days of activation: {most_common_interaction_count}")


In [ ]:
import pandas as pd

# Task 7: Create property_age_category and find most frequent category
property_data['property_age_category'] = pd.cut(property_data['property_age'],
    bins=[0, 1, 5, 10, 20, np.inf],
    labels=['New', 'Less than 5 years', '5 to 10 years', '10 to 20 years', 'More than 20 years'])

most_frequent_age_category = property_data['property_age_category'].mode()[0]
print(f"Most frequent property age category: {most_frequent_age_category}")


In [ ]:
# Task 8: Apartment type with the highest average interactions
avg_interactions_by_type = merged_data.groupby('type')['request_date'].count()
highest_interactions_apartment_type = avg_interactions_by_type.idxmax()
print(f"Apartment type with highest average interactions: {highest_interactions_apartment_type}")


In [ ]:
# Task 9: Amenity with the greatest impact on rent
amenities = ['gym', 'lift', 'swimming_pool']
amenity_impact = {amenity: property_data.groupby(amenity)['rent'].mean().mean() for amenity in amenities}
max_impact_amenity = max(amenity_impact, key=amenity_impact.get)
print(f"Amenity with greatest impact on rent: {max_impact_amenity}")


In [ ]:
# Task 10: Highest interaction counts in top 5 localities with highest average rent
top_5_localities = property_data.groupby('locality')['rent'].mean().nlargest(5).index
top_5_locality_data = merged_data[merged_data['locality'].isin(top_5_localities)]
highest_interaction_locality = top_5_locality_data.groupby('locality')['request_date'].count().idxmax()
print(f"Highest interaction counts for properties in top 5 localities: {highest_interaction_locality}")


In [ ]:
# Task 11: Photo count of the property with highest interactions
photo_data = pd.read_csv('photo_data.csv')

# Clean the photo_urls column to count photos
photo_data['photo_count'] = photo_data['photo_urls'].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
merged_data_with_photos = pd.merge(merged_data, photo_data[['property_id', 'photo_count']], on='property_id')

# Find the property with highest interactions
most_interactions_property = merged_data_with_photos.groupby('property_id')['request_date'].count().idxmax()
photo_count_of_most_interacted_property = merged_data_with_photos[merged_data_with_photos['property_id'] == most_interactions_property]['photo_count'].iloc[0]
print(f"Photo count of the property with highest interactions: {photo_count_of_most_interacted_property}")


In [ ]:
from scipy import stats

# Task 12: Hypothesis test for properties with and without gym
properties_with_gym = merged_data[merged_data['gym'] == 1]
properties_without_gym = merged_data[merged_data['gym'] == 0]

# Perform t-test
t_stat, p_value = stats.ttest_ind(properties_with_gym['request_date'], properties_without_gym['request_date'], equal_var=False)
if p_value < 0.05:
    print("Reject the null hypothesis")
else:
    print("Failed to reject the null hypothesis")


In [ ]:
# Task 13: Create time_category column
merged_data['hour'] = merged_data['request_date'].dt.hour
merged_data['time_category'] = pd.cut(merged_data['hour'],
                                      bins=[0, 5, 11, 17, 23],
                                      labels=['Midnight', 'Morning', 'Afternoon', 'Evening'])

most_frequent_time_category = merged_data['time_category'].mode()[0]
print(f"Most frequent time category: {most_frequent_time_category}")


In [ ]:
# Task 14: Most properties activated on the platform for listing
activation_counts = property_data['activation_date'].value_counts()
most_activated_date = activation_counts.idxmax()
print(f"Most properties activated on the platform: {most_activated_date}")


In [ ]:
# Task 15: Percentage of properties under 'Anyone' lease type
percentage_anyone_lease = (property_data[property_data['lease_type'] == 'Anyone'].shape[0] / property_data.shape[0]) * 100
print(f"Percentage of properties available for lease under 'Anyone' category: {round(percentage_anyone_lease)}%")
